In [ ]:
# AUTOGENERATED! DO NOT EDIT! File to edit: nbs/sgd_shapley.ipynb (unless otherwise specified).

__all__ = ['ncr', 'SGDshapley']

# Cell
# Author: Simon Grah <simon.grah@thalesgroup.com>
#         Vincent Thouvenot <vincent.thouvenot@thalesgroup.com>

# MIT License

# Copyright (c) 2020 Thales Six GTS France

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# Cell
import numpy as np
import pandas as pd
import operator as op
from functools import reduce
import random
from tqdm import tqdm

# Cell
def ncr(n, r):
    """
    Combinatorial computation: number of subsets of size r among n elements
    Efficient algorithm
    """
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer / denom

# Cell
class SGDshapley():
    """
    Estimate the Shapley Values using a Projected Stochastic Gradient algorithm.
    """

    def __init__(self, d, C):
        """
        Calculate internal values for later purposes
        Those elements depend only on the number of features d

        Parameters
        ----------
        d : integer
            Dimension of the problem. The number of features
        """

        # Store in a dictionary for each size k of coalitions
        dict_ω_k = dict() # weights per size k
        dict_L_k = dict() # L-smooth constant per size k
        D = C * np.sqrt(d)
        for k in range(1, d):
            ω_k = (d - 1) / (ncr(d, k) * k * (d - k))
            L_k = ω_k * np.sqrt(k) * (np.sqrt(k) * D + C)
            dict_ω_k.update({k: ω_k})
            dict_L_k.update({k: L_k})
        # Summation of all L per coalition (closed formula)
        sum_L = np.sum([(d-1)/(np.sqrt(k)*(d-k)) * (np.sqrt(k)*D + C) for k in range(1, d)])
        # Probability distributions for sampling new instance
        # Classic SGD
        p = [ncr(d,k) for k in range(1,d)]
        p /= np.sum(p)
        # Importance Sampling proposal q
        q = np.array(list(dict_L_k.values())) * np.array(p)
        q /= np.sum(q)

        # Save internal attributes
        self.d = d
        self.n = 2**d - 2
        self.dict_ω_k = dict_ω_k
        self.dict_L_k = dict_L_k
        self.sum_L = sum_L
        self.p = p
        self.q = q

    def _F_i(self, Φ, x_i, y_i, ω_i):
        """Function value per instance i"""
        res = .5 * self.n * ω_i * (np.dot(x_i, Φ) - y_i)**2
        return res

    def _grad_F_i(self, Φ, x_i, y_i, ω_i):
        """Gradient vector per instance i"""
        res = ω_i * x_i[:,None].dot(x_i[None,:]).dot(Φ) - ω_i * y_i * x_i
        return res

    def _Π_1(self, x, b):
        """Projection Π on convex set K_1"""
        if np.abs((np.sum(x) - b)) <= 1e-6:
            return x
        else:
            return x - (np.sum(x) - b)/len(x)

    def _Π_2(self, x, D):
        """Projection Π on convex set K_2"""
        if np.linalg.norm(x) > D:
            return x * D / np.linalg.norm(x)
        else:
            return x

    def _Dykstra_proj(self, x, D, b, iter_proj=100, epsilon=1e-6):
        """
        Dykstra's algorithm to find orthogonal projection
        onto intersection of convex sets
        """
        xk = x.copy()
        d = len(x)
        pk, qk = np.zeros(d), np.zeros(d)
        for k in range(iter_proj):
            yk = self._Π_2(xk + pk, D)
            pk = xk + pk - yk
            if np.linalg.norm(self._Π_1(yk + qk, b) - xk, 2) <= epsilon:
                break
            else:
                xk = self._Π_1(yk + qk, b)
                qk = yk + qk - xk
        return xk

    def sgd(self, x, fc, ref, n_iter=100, step=.1, step_type="sqrt",
            callback=None, Φ_0=False):
        """
        Stochastic gradient descent algorithm
        The game is defined for an element x, a reference r and function fc

        """
        
        d = self.d
        
                

        # Get general information
        feature_names = list(x.index)
        f_x, f_r = fc(x.values), fc(ref.values)
        v_M = f_x - f_r


        
        n = 2**d - 2
        p = self.p
        dict_ω_k = self.dict_ω_k
        q = self.q
        dict_L_k = self.dict_L_k
        sum_L = self.sum_L

        # Store Shapley Values in a pandas Series
        if Φ_0:
            Φ = Φ_0.copy()
        else:
            Φ = np.zeros(d)
        Φ_storage = np.zeros((n_iter,d))

        # projection onto convex set K by using a simple algorithm
        # Φ = self._Dykstra_proj(Φ, D, v_M, iter_proj, epsilon=1e-6)
        Φ = Φ - (np.sum(Φ) - v_M) / d

        # Sample in advance coalition sizes
        

        for t in tqdm(range(1, n_iter+1)):
            # build x_i
            k = list_k[t-1]
            indexes = np.random.permutation(d)[:k]
            x_i = np.zeros(d)
            x_i[indexes] = 1
            # Compute y_i
            z_S = np.array([x.values[j] if x_i[j] == 1 else ref.values[j] for j in range(d)])
            f_S = fc(z_S)
            y_i = f_S - f_r
            # get weight ω_i
            ω_i = dict_ω_k[k]
            # calculate gradient
            p_i = dict_L_k[k] / sum_L
            grad_i = 1/(p_i) * self._grad_F_i(Φ, x_i, y_i, ω_i)
            # update Φ
            if step_type == "constant":
                Φ = Φ - step * grad_i
            elif step_type == "sqrt":
                Φ = Φ - (step/np.sqrt(t)) * grad_i
            elif step_type == "inverse":
                Φ = Φ - (step/(t)) * grad_i

            # projection onto convex set K
            # Φ = self._Dykstra_proj(Φ, D, v_M, iter_proj, epsilon=1e-6)
            Φ = Φ - (Φ.sum() - v_M) / d

            # update storage of Φ
            Φ_storage[t-1,:] = Φ

            if callback and (t % d == 0):
                callback(pd.Series(np.mean(Φ_storage[:t,:],axis=0), index=feature_names))

        # Average all Φ
        Φ = pd.Series(np.mean(Φ_storage,axis=0), index=feature_names)

        return Φ

In [ ]:
num_players=10
batch_size=5

permutations = np.tile(np.arange(num_players), (batch_size, 1))
arange = np.arange(batch_size)
n = 0
print(arange, arange.shape)
print(permutations, permutations.shape)

In [ ]:
# Begin sampling.
for it in range(5):
    for i in range(batch_size):
        np.random.shuffle(permutations[i])
    S = np.zeros((batch_size, num_players), dtype=bool)
    
    # Sample exogenous (if applicable).

    # Unroll permutations.
    for i in range(num_players):
        S[arange, permutations[:, i]] = 1
        print(S)
    break
    print('loop')

In [ ]:
S[arange, permutations[:, i]]

In [ ]:
permutations

In [ ]:
S[:, permutations[:, i]]

In [ ]:
sgd_shapley=SGDshapley(d=196, C=1)    

In [ ]:
plt.scatter(np.arange(len(np.array(list(sgd_shapley.dict_L_k.values())))), np.array(list(sgd_shapley.dict_L_k.values())))

In [ ]:
plt.hist(np.random.choice(list(range(1, sgd_shapley.d)), size=1000, p=sgd_shapley.q))

In [ ]:
plt.hist(np.random.choice(list(range(1, sgd_shapley.d)), size=1000, p=sgd_shapley.p))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import os
import sys

os.chdir('../')

In [ ]:
import argparse
import glob
from functools import partial
from pathlib import Path

import numpy as np
import torch
from scipy.special import softmax
from tqdm.auto import tqdm

from utils import read_eval_results


class ShapleyValues:
    """For storing and plotting Shapley values."""

    def __init__(self, values, std):
        self.values = values
        self.std = std

    def plot(
        self,
        feature_names=None,
        sort_features=True,
        max_features=np.inf,
        orientation="horizontal",
        error_bars=True,
        color="C0",
        title="Feature Importance",
        title_size=20,
        tick_size=16,
        tick_rotation=None,
        axis_label="",
        label_size=16,
        figsize=(10, 7),
        return_fig=False,
    ):
        """
        Plot Shapley values.

        Args:
        feature_names: list of feature names.
        sort_features: whether to sort features by their Shapley values.
        max_features: number of features to display.
        orientation: horizontal (default) or vertical.
        error_bars: whether to include standard deviation error bars.
        color: bar chart color.
        title: plot title.
        title_size: font size for title.
        tick_size: font size for feature names and numerical values.
        tick_rotation: tick rotation for feature names (vertical plots only).
        label_size: font size for label.
        figsize: figure size (if fig is None).
        return_fig: whether to return matplotlib figure object.
        """
        return plotting.plot(
            self,
            feature_names,
            sort_features,
            max_features,
            orientation,
            error_bars,
            color,
            title,
            title_size,
            tick_size,
            tick_rotation,
            axis_label,
            label_size,
            figsize,
            return_fig,
        )


def default_min_variance_samples(game):
    """Determine min_variance_samples."""
    return 5


def default_variance_batches(num_players, batch_size):
    """
    Determine variance_batches.

    This value tries to ensure that enough samples are included to make A
    approximation non-singular.
    """

    return int(np.ceil(10 * num_players / batch_size))


def calculate_result(A, b, total):
    """Calculate the regression coefficients."""
    num_players = A.shape[1]
    try:
        if len(b.shape) == 2:
            A_inv_one = np.linalg.solve(A, np.ones((num_players, 1)))
        else:
            A_inv_one = np.linalg.solve(A, np.ones(num_players))
        A_inv_vec = np.linalg.solve(A, b)
        values = A_inv_vec - A_inv_one * (
            np.sum(A_inv_vec, axis=0, keepdims=True) - total
        ) / np.sum(A_inv_one)
    except np.linalg.LinAlgError:
        raise ValueError(
            "singular matrix inversion. Consider using larger " "variance_batches"
        )

    return values


def ShapleyRegressionPrecomputed(
    grand_value,
    null_value,
    model_outputs,
    masks,
    num_players,
    batch_size=512,
    detect_convergence=True,
    thresh=0.01,
    n_samples=None,
    paired_sampling=True,
    return_all=False,
    min_variance_samples=None,
    variance_batches=None,
    bar=True,
    verbose=False,
):
    # Verify arguments.
    from tqdm.auto import tqdm

    if min_variance_samples is None:
        min_variance_samples = 5
    else:
        assert isinstance(min_variance_samples, int)
        assert min_variance_samples > 1

    if variance_batches is None:
        variance_batches = default_variance_batches(num_players, batch_size)
    else:
        assert isinstance(variance_batches, int)
        assert variance_batches >= 1

    # Possibly force convergence detection.
    if n_samples is None:
        n_samples = 1e20
        if not detect_convergence:
            detect_convergence = True
            if verbose:
                print("Turning convergence detection on")

    if detect_convergence:
        assert 0 < thresh < 1

    # Weighting kernel (probability of each subset size).
    weights = np.arange(1, num_players)
    weights = 1 / (weights * (num_players - weights))
    weights = weights / np.sum(weights)

    # Calculate null and grand coalitions for constraints.
    null = null_value
    grand = grand_value

    # Calculate difference between grand and null coalitions.
    total = grand - null

    # Set up bar.
    n_loops = int(np.ceil(n_samples / batch_size))
    if bar:
        if detect_convergence:
            bar = tqdm(total=1)
        else:
            bar = tqdm(total=n_loops * batch_size)

    # Setup.
    n = 0
    b = 0
    A = 0
    estimate_list = []

    # For variance estimation.
    A_sample_list = []
    b_sample_list = []

    # For tracking progress.
    var = np.nan * np.ones(num_players)
    if return_all:
        N_list = []
        std_list = []
        val_list = []

    # Begin sampling.
    for it in range(n_loops):
        # Sample subsets.
        # print(subsets.shape)
        S = masks[batch_size * it : batch_size * (it + 1)]
        game_S = model_outputs[batch_size * it : batch_size * (it + 1)]
        #         print("S", S, S.sum(axis=1))
        #         print("game(s)", game_S)
        #         print("game(s)-null", game_S-null)

        A_sample = np.matmul(
            S[:, :, np.newaxis].astype(float), S[:, np.newaxis, :].astype(float)
        )

        b_sample = (S.astype(float).T * (game_S - null)[:, np.newaxis].T).T

        #         print("b", b_sample)
        #         print("variance_batches", variance_batches)

        # Welford's algorithm.
        n += batch_size
        b += np.sum(b_sample - b, axis=0) / n
        A += np.sum(A_sample - A, axis=0) / n

        # Calculate progress.
        values = calculate_result(A, b, total)
        A_sample_list.append(A_sample)
        b_sample_list.append(b_sample)
        if len(A_sample_list) == variance_batches:
            # Aggregate samples for intermediate estimate.
            A_sample = np.concatenate(A_sample_list, axis=0).mean(axis=0)
            b_sample = np.concatenate(b_sample_list, axis=0).mean(axis=0)
            A_sample_list = []
            b_sample_list = []

            # Add new estimate.
            estimate_list.append(calculate_result(A_sample, b_sample, total))

            # Estimate current var.
            # print(len(estimate_list), min_variance_samples)
            if len(estimate_list) >= min_variance_samples:
                var = np.array(estimate_list).var(axis=0)

        # Convergence ratio.
        std = np.sqrt(var * variance_batches / (it + 1))
        ratio = np.max(np.max(std, axis=0) / (values.max(axis=0) - values.min(axis=0)))
        # print("std", var)
        # Print progress message.
        if verbose:
            if detect_convergence:
                print(f"StdDev Ratio = {ratio:.4f} (Converge at {thresh:.4f})")
            else:
                print(f"StdDev Ratio = {ratio:.4f}")

        # Check for convergence.
        if detect_convergence:
            if ratio < thresh:
                if verbose:
                    print("Detected convergence")

                # Skip bar ahead.
                if bar:
                    bar.n = bar.total
                    bar.refresh()
                break

        # Forecast number of iterations required.
        if detect_convergence:
            N_est = (it + 1) * (ratio / thresh) ** 2
            if bar and not np.isnan(N_est):
                bar.n = np.around((it + 1) / N_est, 4)
                bar.refresh()
        elif bar:
            bar.update(batch_size)

        # Save intermediate quantities.
        if return_all:
            val_list.append(values)
            std_list.append(std)
            if detect_convergence:
                N_list.append(N_est)

        # print("size", batch_size*it, len(masks))
        if batch_size * (it + 1) >= len(masks):
            break
    # print(ratio)
    # Return results.
    if return_all:
        # Dictionary for progress tracking.
        iters = (np.arange(it + 1) + 1) * batch_size * (1)
        tracking_dict = {"values": val_list, "std": std_list, "iters": iters}
        if detect_convergence:
            tracking_dict["N_est"] = N_list

        return ShapleyValues(values, std), tracking_dict, ratio
    else:
        return ShapleyValues(values, std), ratio


# def read_eval_results(path):
#     file_set = set(
#         [
#             p
#             for p in glob.glob(str(Path(path) / "*.pt"))
#             if p.split("/")[-1] != "shapley_output.pt"
#         ]
#     )

#     path_grand_null = str(Path(path) / "grand_null.pt")
#     file_set.remove(path_grand_null)

#     file_list = sorted(list(file_set), key=lambda x: x.split("_")[-2])
#     begin_idx = int(file_list[0].split("_")[-2])
#     end_idx = int(file_list[0].split("_")[-1].replace(".pt", ""))
#     step_size = end_idx - begin_idx

#     idx = begin_idx

#     path_eval_list = []
#     while True:
#         path_eval = str(Path(path) / f"mask_eval_{idx}_{idx+step_size}.pt")
#         if path_eval in file_set:
#             file_set.remove(path_eval)
#             path_eval_list.append(path_eval)
#         else:
#             break
#         idx += step_size

#     assert len(file_set) == 0

#     grand_null = torch.load(path_grand_null)
#     eval_list = [torch.load(path_eval) for path_eval in path_eval_list]

#     grand_logits = grand_null["logits"][0]
#     grand_masks = grand_null["masks"][0]
#     null_logits = grand_null["logits"][1]
#     nulll_masks = grand_null["masks"][1]

#     eval_logits = np.concatenate(
#         [eval_value["logits"] for eval_value in eval_list], axis=0
#     )
#     eval_masks = np.concatenate(
#         [eval_value["masks"] for eval_value in eval_list], axis=0
#     )

#     return {
#         "grand": {"logits": grand_logits, "masks": grand_masks},
#         "null": {"logits": null_logits, "masks": nulll_masks},
#         "subsets": {"logits": eval_logits, "masks": eval_masks},
#     }


def get_args():
    """Parse the command line arguments."""
    parser = argparse.ArgumentParser(description="Process some inputs.")

    # Argument for batch size without default
    parser.add_argument(
        "--batch_size", type=int, required=True, help="The batch size for processing."
    )

    # Argument for input path without default
    parser.add_argument(
        "--input_path", type=str, required=True, help="Path to the input directory."
    )

    # Argument for normalization function
    parser.add_argument(
        "--normalize_function",
        type=str,
        choices=["softmax"],
        required=True,
        help="The normalization function to be used. Options: softmax.",
    )

    parser.add_argument(
        "--num_players", type=int, required=True, help="The number of players"
    )

    parser.add_argument(
        "--target_subset_size",
        type=int,
        required=False,
        default=None,
        help="The target subset size",
    )

    return parser.parse_args()

In [ ]:
args = get_args()

In [ ]:
--input_path logs/vitbase_imagenette_surrogate_eval_train/extract_output/train \
--batch_size 512 \
--normalize_function softmax \
--num_players 196

In [ ]:
# Accessing the arguments
batch_size = 512
input_path = "logs/vitbase_imagenette_surrogate_eval_train/extract_output/train"
if "softmax" == "softmax":
    normalize_function = softmax
else:
    raise ValueError("Unsupported normalization function")
num_players = 196
target_subset_size = None

sample_list = glob.glob(str(Path(input_path) / "[0-9]*"))

pbar = tqdm(sample_list)
subsets_output_prev = None

In [ ]:
sample_list

In [ ]:
!ls logs/vitbase_imagenette_surrogate_eval_train/extract_output/train/610

In [ ]:
for sample_path in pbar:
    break

In [ ]:
eval_results = read_eval_results(path=sample_path)

In [ ]:
# AUTOGENERATED! DO NOT EDIT! File to edit: nbs/sgd_shapley.ipynb (unless otherwise specified).

__all__ = ['ncr', 'SGDshapley']

# Cell
# Author: Simon Grah <simon.grah@thalesgroup.com>
#         Vincent Thouvenot <vincent.thouvenot@thalesgroup.com>

# MIT License

# Copyright (c) 2020 Thales Six GTS France

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# Cell
import numpy as np
import pandas as pd
import operator as op
from functools import reduce
import random
from tqdm import tqdm

# Cell
def ncr(n, r):
    """
    Combinatorial computation: number of subsets of size r among n elements
    Efficient algorithm
    """
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer / denom

# Cell
class SGDshapley():
    """
    Estimate the Shapley Values using a Projected Stochastic Gradient algorithm.
    """

    def __init__(self, d, num_classes, C):
        """
        Calculate internal values for later purposes
        Those elements depend only on the number of features d

        Parameters
        ----------
        d : integer
            Dimension of the problem. The number of features
        """

        # Store in a dictionary for each size k of coalitions
        dict_ω_k = dict() # weights per size k
        dict_L_k = dict() # L-smooth constant per size k
        D = C * np.sqrt(d)
        for k in range(1, d):
            ω_k = (d - 1) / (ncr(d, k) * k * (d - k))
            L_k = ω_k * np.sqrt(k) * (np.sqrt(k) * D + C)
            dict_ω_k.update({k: ω_k})
            dict_L_k.update({k: L_k})
        # Summation of all L per coalition (closed formula)
        sum_L = np.sum([(d-1)/(np.sqrt(k)*(d-k)) * (np.sqrt(k)*D + C) for k in range(1, d)])
        # Probability distributions for sampling new instance
        # Classic SGD
        p = [ncr(d,k) for k in range(1,d)]
        print('p', p)
        p /= np.sum(p)
        print(dict_L_k)
        # Importance Sampling proposal q
        q = np.array(list(dict_L_k.values())) * np.array(p)
        q /= np.sum(q)

        # Save internal attributes
        self.num_classes=num_classes
        self.d = d
        self.n = 2**d - 2
        self.dict_ω_k = dict_ω_k
        self.dict_L_k = dict_L_k
        self.sum_L = sum_L
        self.p = p
        self.q = q

    def _F_i(self, Φ, x_i, y_i, ω_i):
        """Function value per instance i"""
        res = .5 * self.n * ω_i * (np.dot(x_i, Φ) - y_i)**2
        return res

    def _grad_F_i(self, Φ, x_i, y_i, ω_i):
        """Gradient vector per instance i"""
        res = ω_i * x_i[:,None].dot(x_i[None,:]).dot(Φ) - ω_i * y_i * x_i
        return res

    def _Π_1(self, x, b):
        """Projection Π on convex set K_1"""
        if np.abs((np.sum(x) - b)) <= 1e-6:
            return x
        else:
            return x - (np.sum(x) - b)/len(x)

    def _Π_2(self, x, D):
        """Projection Π on convex set K_2"""
        if np.linalg.norm(x) > D:
            return x * D / np.linalg.norm(x)
        else:
            return x

    def _Dykstra_proj(self, x, D, b, iter_proj=100, epsilon=1e-6):
        """
        Dykstra's algorithm to find orthogonal projection
        onto intersection of convex sets
        """
        xk = x.copy()
        d = len(x)
        pk, qk = np.zeros(d), np.zeros(d)
        for k in range(iter_proj):
            yk = self._Π_2(xk + pk, D)
            pk = xk + pk - yk
            if np.linalg.norm(self._Π_1(yk + qk, b) - xk, 2) <= epsilon:
                break
            else:
                xk = self._Π_1(yk + qk, b)
                qk = yk + qk - xk
        return xk

    def sgd(self, x, fc, ref, 
            f_x, f_r,
            n_iter=100, step=.1, step_type="sqrt",
            callback=None, Φ_0=False):
        """
        Stochastic gradient descent algorithm
        The game is defined for an element x, a reference r and function fc

        """

        # Get general information
        feature_names = list(x.index)
#         f_x, f_r = fc(x.values), fc(ref.values)
        v_M = f_x - f_r

        num_classes=self.num_classes
        d = self.d
        n = 2**d - 2
        p = self.p
        dict_ω_k = self.dict_ω_k
        q = self.q
        dict_L_k = self.dict_L_k
        sum_L = self.sum_L

        # Store Shapley Values in a pandas Series
        if Φ_0:
            Φ = Φ_0.copy()
        else:
            Φ = np.zeros((d, num_classes))
        Φ_storage = np.zeros((n_iter,d, num_classes))

        # projection onto convex set K by using a simple algorithm
        # Φ = self._Dykstra_proj(Φ, D, v_M, iter_proj, epsilon=1e-6)
        #print(Φ.shape) # (num_features, num_classes)
        #print(v_M)
        Φ = Φ - (np.sum(Φ) - v_M) / d
        #print(Φ.shape)
        #print(Φ.sum(axis=0))

        # Sample in advance coalition sizes
        list_k = np.random.choice(list(range(1, d)), size=n_iter, p=q)
#         print(list_k)
        print(q)
        plt.hist(list_k)

        for t in tqdm(range(1, n_iter+1)):
            # build x_i
            k = list_k[t-1]
            indexes = np.random.permutation(d)[:k]
            x_i = np.zeros(d)
            x_i[indexes] = 1
            # Compute y_i
            z_S = np.array([x.values[j] if x_i[j] == 1 else ref.values[j] for j in range(d)])
            f_S = fc(z_S)
            y_i = f_S - f_r
            # get weight ω_i
            ω_i = dict_ω_k[k]
            # calculate gradient
            p_i = dict_L_k[k] / sum_L
            grad_i = 1/(p_i) * self._grad_F_i(Φ, x_i, y_i, ω_i)
            # update Φ
            if step_type == "constant":
                Φ = Φ - step * grad_i
            elif step_type == "sqrt":
                Φ = Φ - (step/np.sqrt(t)) * grad_i
            elif step_type == "inverse":
                Φ = Φ - (step/(t)) * grad_i

            # projection onto convex set K
            # Φ = self._Dykstra_proj(Φ, D, v_M, iter_proj, epsilon=1e-6)
            Φ = Φ - (Φ.sum() - v_M) / d

            # update storage of Φ
            Φ_storage[t-1,:] = Φ

            if callback and (t % d == 0):
                callback(pd.Series(np.mean(Φ_storage[:t,:],axis=0), index=feature_names))

        # Average all Φ
        Φ = pd.Series(np.mean(Φ_storage,axis=0), index=feature_names)

        return Φ

In [ ]:
sgd_shapley=SGDshapley(d=num_players, 
                       C=partial(normalize_function, axis=0)(eval_results["grand"]["logits"]).max(),
                       num_classes=10)

In [ ]:
sgd_shapley.sgd(x=pd.DataFrame(np.ones(100)).iloc[:,0],
                f_x=grand_value, 
                f_r=null_value, 
                n_iter=1000,
                fc=None, 
                ref=None,)

In [ ]:
for sample_path in pbar:
    eval_results = read_eval_results(path=sample_path)

    grand_value = eval_results["grand"]["logits"]
    if len(grand_value.shape) == 1:
        grand_value = partial(normalize_function, axis=0)(grand_value)
    else:
        raise RuntimeError(f"Not supported grand shape {grand_value.shape}")

    null_value = eval_results["null"]["logits"]
    if len(null_value.shape) == 1:
        null_value = partial(normalize_function, axis=0)(null_value)
    else:
        raise RuntimeError(f"Not supported null shape {null_value.shape}")

    subsets_output = eval_results["subsets"]["logits"]
    if len(subsets_output.shape) == 2:
        subsets_output = partial(normalize_function, axis=1)(subsets_output)
    else:
        raise RuntimeError(
            f"Not supported subset outputs shape {subsets_output.shape}"
        )
    if subsets_output_prev is not None and len(subsets_output) != len(
        subsets_output_prev
    ):
        print(
            "subsets_output_prev",
            subsets_output_prev.shape,
            "subsets_output",
            subsets_output.shape,
            "sample_path",
            sample_path,
        )

    subsets = eval_results["subsets"]["masks"]

    assert (
        subsets_output.shape[1] == grand_value.shape[0] == null_value.shape[0]
    ), f"Num of classes mismatch {subsets_output.shape[1]} , {grand_value.shape[0]} , {null_value.shape[0]}"
    assert (
        subsets.shape[1] == num_players
    ), f"Num of players mismatch {subsets.shape[1]} != {num_players}"
    break

In [ ]:
pd.DataFrame(np.ones(100)).iloc[:,0]

In [ ]:
sgd_shapley.sgd(x=1,fc=None, ref=None,)

In [ ]:
x, fc, ref, n_iter=100, step=.1, step_type="sqrt",
            callback=None, Φ_0=False

In [ ]:
sgd_shapley=SGDshapley(d=num_players, 
                       C=0.9)

In [ ]:
sgd_shapley

In [ ]:
sgd_shapley.sum_L

In [ ]:
        null_value = partial(normalize_function, axis=0)(null_value)
    else:
        raise RuntimeError(f"Not supported null shape {null_value.shape}")

    subsets_output = eval_results["subsets"]["logits"]
    if len(subsets_output.shape) == 2:
        subsets_output = partial(normalize_function, axis=1)(subsets_output)

In [ ]:
partial(normalize_function, axis=0)(eval_results["grand"]["logits"]).max()

In [ ]:
partial(normalize_function, axis=0)(eval_results["grand"]["logits"])

In [ ]:
partial(normalize_function, axis=1)(eval_results["subsets"]["logits"]).max()

In [ ]:
eval_results["subsets"]["logits"]

In [ ]:
for sample_path in pbar:
    eval_results = read_eval_results(path=sample_path)

    grand_value = eval_results["grand"]["logits"]
    if len(grand_value.shape) == 1:
        grand_value = partial(normalize_function, axis=0)(grand_value)
    else:
        raise RuntimeError(f"Not supported grand shape {grand_value.shape}")

    null_value = eval_results["null"]["logits"]
    if len(null_value.shape) == 1:
        null_value = partial(normalize_function, axis=0)(null_value)
    else:
        raise RuntimeError(f"Not supported null shape {null_value.shape}")

    subsets_output = eval_results["subsets"]["logits"]
    if len(subsets_output.shape) == 2:
        subsets_output = partial(normalize_function, axis=1)(subsets_output)
    else:
        raise RuntimeError(
            f"Not supported subset outputs shape {subsets_output.shape}"
        )
    if subsets_output_prev is not None and len(subsets_output) != len(
        subsets_output_prev
    ):
        print(
            "subsets_output_prev",
            subsets_output_prev.shape,
            "subsets_output",
            subsets_output.shape,
            "sample_path",
            sample_path,
        )

    subsets = eval_results["subsets"]["masks"]

    assert (
        subsets_output.shape[1] == grand_value.shape[0] == null_value.shape[0]
    ), f"Num of classes mismatch {subsets_output.shape[1]} , {grand_value.shape[0]} , {null_value.shape[0]}"
    assert (
        subsets.shape[1] == num_players
    ), f"Num of players mismatch {subsets.shape[1]} != {num_players}"
    sdsd

In [ ]:
    if target_subset_size is None:
        _, tracking_dict, ratio = ShapleyRegressionPrecomputed(
            grand_value=grand_value,
            null_value=null_value,
            model_outputs=subsets_output,
            masks=subsets,
            batch_size=batch_size,
            num_players=num_players,
            variance_batches=2,
            min_variance_samples=2,
            return_all=True,
            bar=False,
        )

#         torch.save(
#             obj=tracking_dict, f=str(Path(sample_path) / "shapley_output.pt")
#         )
    else:
        for subset_group_idx in range(len(subsets_output) // target_subset_size):
            _, tracking_dict, ratio = ShapleyRegressionPrecomputed(
                grand_value=grand_value,
                null_value=null_value,
                model_outputs=subsets_output[
                    target_subset_size
                    * subset_group_idx : target_subset_size
                    * (subset_group_idx + 1)
                ],
                masks=subsets[
                    target_subset_size
                    * subset_group_idx : target_subset_size
                    * (subset_group_idx + 1)
                ],
                batch_size=batch_size,
                num_players=num_players,
                variance_batches=2,
                min_variance_samples=2,
                return_all=True,
                bar=False,
            )

#             torch.save(
#                 obj=tracking_dict,
#                 f=str(
#                     Path(sample_path)
#                     / f"shapley_output_{target_subset_size}_{subset_group_idx}.pt"
#                 ),
#             )

    pbar.set_postfix(
        ratio=ratio,
        num_masks=len(subsets),
        refresh=True,
    )